In [12]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from keras import backend as K
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier

#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [3]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [4]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [5]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [6]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples].reshape(15998,1000,1)
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:].reshape(3999,1000,1)
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997,)


In [7]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


In [8]:
def no_embedding(fl1=16, fl2=16, fl3=16, dl=16, optimizer= 'RMSprop', kl = 5, layer =1 ):
    inp =  Input(shape=(1000, 1))
    if (layer == 1):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
    elif (layer == 2):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
    else:
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(inp)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl3, kernel_size = kl, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(units = dl, activation='relu')(x)
    preds = Dense(1, activation='tanh')(x)
    model = Model(inp, preds)
    model.compile(loss= 'binary_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
   
    return model

In [9]:
def kera_trainable_params(model):
    trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])) 
    return trainable_count

In [10]:
def loss_edit(out):
    loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in out.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in out.history.keys() if 'acc' in s and 'val' in s]
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
    for l in loss_list:
        train_loss .append(out.history[l])
    for l in val_loss_list:
        val_loss .append(out.history[l])
    for l in acc_list:
        train_acc.append(out.history[l])
    for l in val_acc_list:
        val_acc .append(out.history[l])
    line  = { 'train_loss': reduce(operator.concat, train_loss)[-1],'val_loss': reduce(operator.concat, val_loss)[-1], \
             'train_acc': reduce(operator.concat, train_acc)[-1], 'val_acc': reduce(operator.concat, val_acc)[-1] }
    return (line)

In [ ]:
# bounds for hyper-parameters in mnist model
# the bounds dict should be in order of continuous type and then discrete type
# create model
# define the grid search parameters
layers = [1,2,3]
fl1s = [16,32,64, 128]
fl2s = [16,32,64, 128]
fl3s = [16,32,64,128]
kls = [5]
dls = [16,32,64,128]
#sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
optimizers = ['Nadam', 'Adam', 'RMSprop', 'SGD'] #, 'Adadelta', 'Adam', 'Adamax', 'Nadam']
loss_data = []
params = []
trainable_count = []
for fl1, fl2, fl3, kl, dl, optimizer,layer in product(fl1s,fl2s,fl3s,kls,dls,optimizers,layers):
    kwargs = dict(fl1=fl1, fl2= fl2, fl3=fl3, kl=kl, dl=dl, optimizer= ''.join(optimizer), layer=layer)
    params.append(kwargs)
    model = no_embedding(**kwargs)
    trainable_count.append(kera_trainable_params(model))
    history = model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
    loss_data.append(loss_edit(history))
    print(loss_edit(history))

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 10s 635us/step - loss: 8.1733 - acc: 0.1121 - val_loss: 8.1699 - val_acc: 0.0545
Epoch 2/2
15998/15998 [==============================] - 8s 499us/step - loss: 8.2716 - acc: 0.0504 - val_loss: 8.1699 - val_acc: 0.0545
{'train_loss': 8.27163170385188, 'val_loss': 8.169887369470198, 'train_acc': 0.050381297665002094, 'val_acc': 0.05451362843318533}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 10s 626us/step - loss: 3.7423 - acc: 0.3288 - val_loss: 2.3315 - val_acc: 0.4366
Epoch 2/2
15998/15998 [==============================] - 9s 579us/step - loss: 2.2760 - acc: 0.4090 - val_loss: 2.6063 - val_acc: 0.3958
{'train_loss': 2.2760136232687276, 'val_loss': 2.6062887793751055, 'train_acc': 0.408988623581673, 'val_acc': 0.39584896234489436}
Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==================

In [ ]:
params_pd = pd.DataFrame(params)
loss_pd= pd.DataFrame(loss_data)
trainable_pd = pd.DataFrame({'trainable_params': trainable_count})

In [26]:
no_embeddings = pd.concat([params_pd, loss_pd, trainable_pd], axis=1, join='inner')
no_embeddings.to_csv('no_embeddings_models.csv', index=False)


In [13]:
no_embeddings = pd.read_csv('no_embeddings_models.csv')
no_embeddings_sort = no_embeddings.sort_values(['val_loss'])
no_embeddings_sort

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss,trainable_params
71,32,16,16,32,5,3,Nadam,0.572384,0.673456,0.576144,0.661850,5106
491,16,16,64,64,5,3,Nadam,0.563133,0.670764,0.570393,0.663311,26898
515,64,16,64,64,5,3,Nadam,0.558320,0.687657,0.569142,0.663556,30114
53,16,16,16,32,5,3,Adam,0.570259,0.665551,0.562641,0.665051,4546
437,16,16,64,32,5,3,Adam,0.569884,0.679560,0.586397,0.669365,16114
353,32,16,32,128,5,3,Adam,0.576072,0.675641,0.582896,0.669398,27490
221,64,16,32,16,5,3,Adam,0.545631,0.679958,0.560140,0.669625,6482
275,64,16,32,32,5,3,Nadam,0.554444,0.670132,0.559890,0.669899,10082
442,16,16,64,32,5,2,Nadam,0.554882,0.674300,0.570893,0.669979,6354
2,16,16,16,16,5,3,RMSprop,0.566571,0.680231,0.587897,0.670605,2994
